In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import io # Input/Output Module
import os # OS interfaces
import cv2 # OpenCV package
import numpy as np # linear algebra
import keras
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from urllib import request # module for opening HTTP requests
from matplotlib import pyplot as plt # Plotting library

<div style="width:100%; height:140px">
    <img src="https://www.kuleuven.be/internationaal/thinktank/fotos-en-logos/ku-leuven-logo.png/image_preview" width = 300px, heigh = auto align=left>
</div>


KUL H02A5a Computer Vision: Group Assignment 1
---------------------------------------------------------------
Student numbers: <span style="color:red">r0703889, r0909802, r0716758, r0916443, r0822692</span>.

The goal of this assignment is to explore more advanced techniques for constructing features that better describe objects of interest and to perform face recognition using these features. This assignment will be delivered in groups of 5 (either composed by you or randomly assigned by your TA's).

In this assignment you are a group of computer vision experts that have been invited to ECCV 2021 to do a tutorial about  "Feature representations, then and now". To prepare the tutorial you are asked to participate in a kaggle competition and to release a notebook that can be easily studied by the tutorial participants. Your target audience is: (master) students who want to get a first hands-on introduction to the techniques that you apply.

---------------------------------------------------------------
This notebook is structured as follows:
0. Data loading & Preprocessing
1. Feature Representations
2. Evaluation Metrics 
3. Classifiers
4. Experiments
5. Publishing best results
6. Discussion

Make sure that your notebook is **self-contained** and **fully documented**. Walk us through all steps of your code. Treat your notebook as a tutorial for students who need to get a first hands-on introduction to the techniques that you apply. Provide strong arguments for the design choices that you made and what insights you got from your experiments. Make use of the *Group assignment* forum/discussion board on Toledo if you have any questions.

Fill in your student numbers above and get to it! Good luck! 


<div class="alert alert-block alert-info">
<b>NOTE:</b> This notebook is just a example/template, feel free to adjust in any way you please! Just keep things organised and document accordingly!
</div>

<div class="alert alert-block alert-info">
<b>NOTE:</b> Clearly indicate the improvements that you make!!! You can for instance use titles like: <i>3.1. Improvement: Non-linear SVM with RBF Kernel.<i>
</div>
    
---------------------------------------------------------------
# 0. Data loading & Preprocessing

## 0.1. Loading data
The training set is many times smaller than the test set and this might strike you as odd, however, this is close to a real world scenario where your system might be put through daily use! In this session we will try to do the best we can with the data that we've got! 

In [ ]:
os.listdir('/kaggle/input/computer-vision-assignment/kul-h02a5a-computer-vision-ga1-2022')

In [ ]:
# Input data files are available in the read-only "../input/" directory

train = pd.read_csv(
    '/kaggle/input/computer-vision-assignment/kul-h02a5a-computer-vision-ga1-2022/train_set.csv', index_col = 0)
train.index = train.index.rename('id')

test = pd.read_csv(
    '/kaggle/input/computer-vision-assignment/kul-h02a5a-computer-vision-ga1-2022/test_set.csv', index_col = 0)
test.index = test.index.rename('id')

# read the images as numpy arrays and store in "img" column
train['img'] = [cv2.cvtColor(np.load('/kaggle/input/computer-vision-assignment/kul-h02a5a-computer-vision-ga1-2022/train/train_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
                for index, row in train.iterrows()]

test['img'] = [cv2.cvtColor(np.load('/kaggle/input/computer-vision-assignment/kul-h02a5a-computer-vision-ga1-2022/test/test_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
                for index, row in test.iterrows()]
  

train_size, test_size = len(train),len(test)

"The training set contains {} examples, the test set contains {} examples.".format(train_size, test_size)

*Note: this dataset is a subset of the* [*VGG face dataset*](https://www.robots.ox.ac.uk/~vgg/data/vgg_face/).

## 0.2. A first look
Let's have a look at the data columns and class distribution.

In [ ]:
# The training set contains an identifier, name, image information and class label
train.head()

In [ ]:
# The test set only contains an identifier and corresponding image information.

test.head(1)

In [ ]:
# The class distribution in the training set:
train.groupby('name').agg({'img':'count', 'class': 'max'})

Note that **Jesse is assigned the classification label 1**, and **Mila is assigned the classification label 2**. The dataset also contains 20 images of **look alikes (assigned classification label 0)** and the raw images. 

## 0.3. Preprocess data
### 0.3.1 Example: HAAR face detector
In this example we use the [HAAR feature based cascade classifiers](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html) to detect faces, then the faces are resized so that they all have the same shape. If there are multiple faces in an image, we only take the first one. 

<div class="alert alert-block alert-info"> <b>NOTE:</b> You can write temporary files to <code>/kaggle/temp/</code> or <code>../../tmp</code>, but they won't be saved outside of the current session
</div>


In [ ]:
class HAARPreprocessor():
    """Preprocessing pipeline built around HAAR feature based cascade classifiers. """
    
    def __init__(self, path, face_size):
        self.face_size = face_size
        file_path = os.path.join(path, "haarcascade_frontalface_default.xml")
        if not os.path.exists(file_path): 
            if not os.path.exists(path):
                os.mkdir(path)
            self.download_model(file_path)
        
        self.classifier = cv2.CascadeClassifier(file_path)
  
    def download_model(self, path):
        url = "https://raw.githubusercontent.com/opencv/opencv/master/data/"\
            "haarcascades/haarcascade_frontalface_default.xml"
        
        with request.urlopen(url) as r, open(path, 'wb') as f:
            f.write(r.read())
            
    def detect_faces(self, img):
        """Detect all faces in an image."""
        
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return self.classifier.detectMultiScale(
            img_gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
    def extract_faces(self, img):
        """Returns all faces (cropped) in an image."""
        
        faces = self.detect_faces(img)

        return [img[y:y+h, x:x+w] for (x, y, w, h) in faces]
    
    def preprocess(self, data_row):
        faces = self.extract_faces(data_row['img'])
        
        # if no faces were found, return None
        if len(faces) == 0:
            nan_img = np.empty(self.face_size + (3,))
            nan_img[:] = np.nan
            return nan_img
        
        # only return the first face
        return cv2.resize(faces[0], self.face_size, interpolation = cv2.INTER_AREA)
            
    def __call__(self, data):
        return np.stack([self.preprocess(row) for _, row in data.iterrows()]).astype(int)

**Visualise**

Let's plot a few examples.

In [ ]:
# parameter to play with 
FACE_SIZE = (100, 100)

def plot_image_sequence(data, n, imgs_per_row=7):
    n_rows = 1 + int(n/(imgs_per_row+1))
    n_cols = min(imgs_per_row, n)

    f,ax = plt.subplots(n_rows,n_cols, figsize=(10*n_cols,10*n_rows))
    for i in range(n):
        if n == 1:
            ax.imshow(data[i])
        elif n_rows > 1:
            ax[int(i/imgs_per_row),int(i%imgs_per_row)].imshow(data[i])
        else:
            ax[int(i%n)].imshow(data[i])
    plt.show()

    
#preprocessed data 
preprocessor = HAARPreprocessor(path = '../../tmp', face_size=FACE_SIZE)

train_X, train_y = preprocessor(train), train['class'].values
test_X = preprocessor(test)



In [ ]:
# plot faces of Michael and Sarah

plot_image_sequence(train_X[train_y == 0], n=20, imgs_per_row=10)

In [ ]:
# plot faces of Jesse

plot_image_sequence(train_X[train_y == 1], n=30, imgs_per_row=10)

In [ ]:
# plot faces of Mila

plot_image_sequence(train_X[train_y == 2], n=30, imgs_per_row=10)

## 0.4. Store Preprocessed data (optional)
<div class="alert alert-block alert-info">
<b>NOTE:</b> You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All". Feel free to use this to store intermediary results.
</div>

In [ ]:
# save preprocessed data
# prep_path = '/kaggle/working/prepped_data/'
# if not os.path.exists(prep_path):
#     os.mkdir(prep_path)
    
# np.save(os.path.join(prep_path, 'train_X.npy'), train_X)
# np.save(os.path.join(prep_path, 'train_y.npy'), train_y)
# np.save(os.path.join(prep_path, 'test_X.npy'), test_X)

# load preprocessed data
# prep_path = '/kaggle/working/prepped_data/'
# if not os.path.exists(prep_path):
#     os.mkdir(prep_path)
# train_X = np.load(os.path.join(prep_path, 'train_X.npy'))
# train_y = np.load(os.path.join(prep_path, 'train_y.npy'))
# test_X = np.load(os.path.join(prep_path, 'test_X.npy'))

Now we are ready to rock!

# 1. Feature Representations
## 1.0. Example: Identify feature extractor
Our example feature extractor doesn't actually do anything... It just returns the input:
$$
\forall x : f(x) = x.
$$

It does make for a good placeholder and baseclass ;).

In [ ]:
class IdentityFeatureExtractor:
    """A simple function that returns the input"""
    
    def transform(self, X):
        return X
    
    def __call__(self, X):
        return self.transform(X)

In [ ]:
class Preprocessing:
    """Preprocessing of data for future"""
    
    def transform(self, X):
        nb_train_images = 80
        M = np.array(np.zeros([nb_train_images,10000]))
        for n in range(nb_train_images):
            Im = train_X[train_y][n]
            Im = rgb2gray(Im)
            A = np.asarray(Im).reshape(-1)
            M[n,:] = A
            return M
        
        
    def __call__(self, X):
        return self.transform(X)

## 1.1. Baseline 1: HOG feature extractor/Scale Invariant Feature Transform

HOG and SIFT are both very similar feature descriptors. 
However, the HOG feature extractor is used more frequently in classification problems than SIFT. The latter is used more in identificating specific objects. By this reason, we decided to implement the HOG feature extractor.


The HOG feature extractor counts occurences of gradient orientations in a localized portion of an image. The purpose of this technique to receive a better objects detection by focusing on the structure/shape of an object. This technique is widely used in computer vision and image processing, and was introducedin a research paper of Dala and Triggs in 2005.
This technique returns mostly sligtly better results than other edge descriptors because it also uses the magnitude and angle gradient to compute features, from which it generates an histogram.


In [ ]:
#importing required libraries
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
from skimage import data
from skimage.color import (rgb2gray)
import matplotlib.pyplot as plt

In [ ]:
class HOGFeatureExtractor(IdentityFeatureExtractor):
    """
    https://www.thepythoncode.com/article/hog-feature-extraction-in-python

    """
    
    def __init__(self, X):
        self.X = X

    def save_extract(self):
        """
         TODO: DOES NOT EXPORT ANYMORE - PROBLEM!! 
         
         Outputs the images converted by HOG extractor
         
         Returns
         -------
         list_img : Array of arrays representing the Hog_images
        """
        n = train_X[train_y].shape
        imgs_per_row = 10
        n_rows = 1 + int(n[0]/(imgs_per_row+1))
        n_cols = min(imgs_per_row, n[0])
        list_img = []
        
        for i in range(n[0]):
            img = train_X[train_y][i]
            fd, hog_image = hog(img, orientations=10, pixels_per_cell=(5, 5),  cells_per_block=(2, 2), visualize=True, multichannel=True)
        
            prep_path = '/kaggle/working/prepped_data/'
            if not os.path.exists(prep_path):
                os.mkdir(prep_path)
            
            np.save(os.path.join(prep_path, 'train_X_HOG_{}.npy'.format(i)), hog_image)
            list_img.append(hog_image)
        
        return list_img

    def plot_extract(self):
        n = train_X[train_y].shape
        imgs_per_row = 10
        n_rows = 1 + int(n[0]/(imgs_per_row+1))
        n_cols = min(imgs_per_row, n[0])
        list_img = []

        for i in range(n[0]):
            img = train_X[train_y][i]
            fd, hog_image = hog(img, orientations=10, pixels_per_cell=(5, 5), cells_per_block=(2, 2), visualize=True, multichannel=True)
            list_img.append(hog_image)

        # creating hog features
        f,ax = plt.subplots(n_rows,n_cols, figsize=(10*n_cols,10*n_rows))

        for i in range(n[0]):
            if n[0] == 1:
                ax.imshow(list_img[i], cmap="gray")
            elif n_rows > 1:
                ax[int(i/imgs_per_row),int(i%imgs_per_row)].imshow(list_img[i], cmap="gray")
            else:
                ax[int(i%n[0])].imshow(list_img[i], cmap="gray")

        plt.show()  


    def transform(self, X):
        raise NotImplmentedError

In [ ]:
hog_model = HOGFeatureExtractor(4)

# Save the extracted data to /kaggle/working
hog_model.save_extract()


# plot the extracted data as hog
hog_model.plot_extract()

### 1.1.1. t-SNE Plots
t-Distributed Stochastic Neighbor Embedding (t-SNE) is not a mathematical technique as PCA, but a probabilistic one. It minimizes the divergence between two distributions: a distribution that measures pairwise similarities of the input objects and a distribution that measures pairwise similarities of the corresponding low-dimensional points in the embedding [HERSCHRIJVEN - COPYRIGHT]

In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns

In [ ]:
# save preprocessed data
list_img = hog_model.save_extract()

prep_path = '/kaggle/working/prepped_data/'
if not os.path.exists(prep_path):
     os.mkdir(prep_path)
    
np.save(os.path.join(prep_path, 'train_X_HOG.npy'), list_img)
#np.save(os.path.join(prep_path, 'train_y.npy'), train_y)
#np.save(os.path.join(prep_path, 'test_X.npy'), test_X)

In [ ]:
# Dataset
# https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b

# Attach img to dataframe
#train.head()
#train.index = train.index.rename('id')

#train['img_HOG'] = [cv2.cvtColor(np.load('/content/drive/MyDrive/kaggle/working/prepped_data/train_X_HOG_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
#                for index, row in train.iterrows()]


train['img_hog'] = [np.load('/kaggle/working/prepped_data/train_X_HOG_{}.npy'.format(index), allow_pickle=False) 
                for index, row in train.iterrows()]


train.head()


In [ ]:
df = train['img_hog']

nb_train_images = 80

# lowers dimension from 3 to 2
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

M = np.array(np.zeros([nb_train_images,10000]))
for n in range(nb_train_images):
    Im = train_X[train_y][n]
    Im = rgb2gray(Im)
    A = np.asarray(Im).reshape(-1)
    M[n,:] = A


tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, random_state=1)

tsne_results = tsne.fit_transform(M)
TSNE1 = tsne_results[:,0].tolist()
TSNE2 = tsne_results[:,1].tolist()

sns.scatterplot(
    x=TSNE1, y=TSNE2, hue=train['class'], palette=sns.color_palette("hls", 3), data=df, legend="full"
)


### 1.1.2. Discussion
#### a) How to make your descriptor behave well in different circumstances (e.g. image with different lighting)?
By simply **normalizing** the images with a 'L2' parameter, we **avoid brightness, contrast or other illumination effects**. This was done by adding `block_norm= ‘L2’` as a parameter of the HOG feature extractor. We could also have used L1 and L2-Hys as volunteering parameters. L2-Hys is sometimes used to reduce noise with the help of a L2-norm.


#### b) Visualize Feature Representation: [TODO]
t-SNE : This method classifies the images into clusters with similarities. Given N points -  $x_1,x_2,…,x_N$ in any dimension, t-SNE will compute the probability  pi,
j  the probability of similarity of  $x_i$  and  $x_j$ . The similarity is calculated based on the t-distribution. 
In practice given a cluster of N points t-SNE helps in finding clusters in which there exists similarity in the data points.

The parameters for t-SNE are number of components =2, perplexity=40 which governs the number of nearest neighbours, learning rate=200, if this is too high the data will look like a ball with equispaced nearest neighbours.     
number of iterations for optimization = 300. We also chose verbose = 1. The other various parameters were chosen to be default namely early_exaggeration, n_iter_without_progress, min_grad_norm,random_state=None, method='barnes_hut'.


#### c) How does this feature compare to your previous grabbing task in the individual assignment?
The HOG feature extractor is a gradient based method and is generally known as a more robust method than the one we used in the individual assignment. In the individual assignment, the object grabbing was based on a **threshold of a particular colour**. The HOG feature extractor in contrary, uses the **magnitude and angle gradient** to compute features. It is computed by dividing an entire image into smaller cells an summing up their gradients over every pixel within each cell in an image.


#### d) Did you need specific pre-processing steps before computing these feature descriptors on your images (which ones and why)?
First of all, we convert the images to a **grayscale** as a basement for the following steps. Then, before processing, the images have been **resized**. In the paper of Dalal and Triggs, dimensions 128x64 were advised. However, the size depends on the application. In the notebook, the images were already resized to 100x100 pixels. We decided to resize the images to 80x80, so it would divisible by 4, 8 and 16. 
Further, we **normalized** the images with a 'L2' parameter, **to avoid brightness, contrast or other illumination effects**. Other possible paramaters we could have used are L1 and L2-Hys. Finally, in order to take a few neighboring pixels into account instead of single pixel values as its true value, we performed a **Gaussian filtering**.

#### e) Did the visualisation show good discriminative and robustness properties? [TODO]
Yes the t-SNE is able to show 3 major clusters of Jesse, Mila and the look alike. The HOG is able to show the major features of a face like nose, boundary, eyes etc. This is only possible for 
images with distinct values in pixels. For the black images which are outliers this is not possible.


#### f) Discussing missing values

In class 1 and 2, several black images appear. This means that the black images only contain 0 values, thus no information. Consequently, we can remove those images since they do not help our algorithm in classifying the faces correctly. 

Thereby, a few images which do not contain a face, are added to the training set.
In statistics, it is advised noy to throw away outliers unless we surely know that they are measurement errors. This is why we decided to keep these images in our dataset. Thereby, they can probably help us not to overfit certain parameters of a face. 

## 1.2. Baseline 2: PCA feature extractor
...

In [ ]:
class PCAFeatureExtractor(IdentityFeatureExtractor):
    """TODO: this feature extractor is under construction"""
    
    def __init__(self, n_components):
        self.n_components = n_components
        
    def transform(self, X):
        raise NotImplmentedError
        
    def inverse_transform(self, X):
        raise NotImplmentedError

### 1.2.1. Eigenface Plots
...

### 1.2.2. Feature Space Plots
...

### 1.2.3. Discussion
...

# 2. Evaluation Metrics
## 2.0. Example: Accuracy
As example metric we take the accuracy. Informally, accuracy is the proportion of correct predictions over the total amount of predictions. It is used a lot in classification but it certainly has its disadvantages...

In [ ]:
from sklearn.metrics import accuracy_score

# 3. Classifiers
## 3.0. Example: The *'not so smart'* classifier
This random classifier is not very complicated. It makes predictions at random, based on the distribution obseved in the training set. **It thus assumes** that the class labels of the test set will be distributed similarly to the training set.

In [ ]:
class RandomClassificationModel:
    """Random classifier, draws a random sample based on class distribution observed 
    during training."""
    
    def fit(self, X, y):
        """Adjusts the class ratio instance variable to the one observed in y. 

        Parameters
        ----------
        X : tensor
            Training set
        y : array
            Training set labels

        Returns
        -------
        self : RandomClassificationModel
        """
        
        self.classes, self.class_ratio = np.unique(y, return_counts=True)
        self.class_ratio = self.class_ratio / self.class_ratio.sum()
        return self
        
    def predict(self, X):
        """Samples labels for the input data. 

        Parameters
        ----------
        X : tensor
            dataset
            
        Returns
        -------
        y_star : array
            'Predicted' labels
        """

        np.random.seed(0)
        return np.random.choice(self.classes, size = X.shape[0], p=self.class_ratio)
    
    def __call__(self, X):
        return self.predict(X)
    

## 3.1. Baseline 1: My favorite classifier
...

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X = data[["img_hog"]]
y = data[["class"]]

nb_train_images = 80

# 1 Train-validation set
X_train, X_test, y_train, y_test = train_test_split(prepro(), y, test_size=0.2, random_state=0)

# show how it looks like
print('Train set:', X_train.shape)
print('Test set:', X_test.shape)

print(X.shape)
print(y.shape)

In [ ]:
X = data[["img_hog"]]
y = data[["class"]]



def prepro():
    nb_train_images = 80
    
    M = np.array(np.zeros([nb_train_images,10000]))
    for n in range(nb_train_images):
        Im = train_X[train_y][n]
        Im = rgb2gray(Im)
        A = np.asarray(Im).reshape(-1)
        M[n,:] = A
        return M

# Full data: name, class, img, img_hog
#data = train
#print(data)

# 1 Train-validation set
X_train, X_test, y_train, y_test = train_test_split(prepro(), y, test_size=0.2, random_state=0)

print('Train set:', X_train.shape)
print('Test set:', X_test.shape)

print(X.shape)
print(y.shape)

DT_model = DecisionTreeClassifier()

DT_model.fit(X_train, y_train)
DT_predict = DT_model.predict(X_test)
DT_score = DT_model.score(X_train,y_train)

DT_score

In [ ]:
class DecisionTree:
    """TODO: this classifier is under construction."""
    def __init__(self):
        self.model = DecisionTreeClassifier()
        
    def fit(self, X, y):
        self.model.fit(X, y)
                
    def predict(self, X):
        self.predict = self.model.predict(X)
        
    def score(self, X, y):
        return self.model.score(X, y)

In [ ]:
DT = DecisionTree()
DT.fit(X_train, y_train)
DT.predict(X_test)
DT.score(X_train, y_train)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X = data[["img_hog"]]
y = data[["class"]]

nb_train_images = 80

def prepro():
    M = np.array(np.zeros([nb_train_images,10000]))
    for n in range(nb_train_images):
        Im = train_X[train_y][n]
        Im = rgb2gray(Im)
        A = np.asarray(Im).reshape(-1)
        M[n,:] = A
        return M

# Full data: name, class, img, img_hog
data = train
#print(data)

# 1 Train-validation set
X_train, X_test, y_train, y_test = train_test_split(prepro(), y, test_size=0.2, random_state=0)

print('Train set:', X_train.shape)
print('Test set:', X_test.shape)

print(X.shape)
print(y.shape)

DT_model = DecisionTreeClassifier()

DT_model.fit(X_train, y_train)
DT_predict = DT_model.predict(X_test)
DT_score = DT_model.score(X_train,y_train)

DT_score

In [ ]:
class FavoriteClassificationModel:
    """TODO: this classifier is under construction."""
    
    def fit(self, X, y):
        raise NotImplmentedError
        
    def predict(self, X):
        raise NotImplmentedError

# 4. Experiments
<div class="alert alert-block alert-info"> <b>NOTE:</b> Do <i>NOT</i> use this section to keep track of every little change you make in your code! Instead, highlight the most important findings and the major (best) pipelines that you've discovered.  
</div>
<br>

## 4.0. Example: basic pipeline
The basic pipeline takes any input and samples a label based on the class label distribution of the training set. As expected the performance is very poor, predicting approximately 1/4 correctly on the training set. There is a lot of room for improvement but this is left to you ;). 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# FEATURE EXTRACTOR WILL SELECT A FEW FEATURES (..) ON WHICH WE WILL TRAIN. NOW
# WWe are training on all the data.
feature_extractor = IdentityFeatureExtractor() 

classifier = DecisionTree()

classifier.fit(X_train, y_train)
#classifier.predict(X_test)
#classifier.score(X_train, y_train)

#print(classifier)

# train the model on the features
#classifier.fit(feature_extractor(train_X), train_y)
classifier.fit(feature_extractor(X_train), y_train)
#classifier.fit(M, train_y)

# model/final pipeline
model = lambda X: classifier(feature_extractor(X))

In [ ]:
# evaluate performance of the model on the training set
#train_y_star = model(train_X)
train_y_star = model(X_train)

"The performance on the training set is {:.2f}. This however, does not tell us much about the actual performance (generalisability).".format(
    accuracy_score(train_y, train_y_star))

In [ ]:
# predict the labels for the test set 
test_y_star = model(X_test)

# 5. Publishing best results

In [ ]:
submission = test.copy().drop('img', axis = 1)
submission['class'] = test_y_star

submission

In [ ]:
submission.to_csv('submission.csv')

# 6. Discussion
...

In summary we contributed the following: 
* 
